In [138]:
import os
import ffmpeg
from datetime import timedelta
import yaml
import pandas as pd
import numpy as np
from itertools import combinations
from itertools import product

In [140]:
df = pd.read_csv('D:\\GITHUB\\Data_CEP\\Exploration\\dataframe_expanded.csv')

In [142]:
#First classification layer: Sort pass (Normal, Head, Hand)
df_pass = df[(df['primary_type'] == 'pass') & (df['cross'] == 0)]
print(len(df_pass))
print(len(df_pass[df_pass['head_pass'] == 1]))
print(len(df_pass[df_pass['hand_pass'] == 1]))
df_pass['normal_pass'] = 1  # Initialize the column with default values (e.g., 1)
df_pass.loc[(df_pass['head_pass'] == 1) | (df_pass['hand_pass'] == 1), 'normal_pass'] = 0
print(len(df_pass[df_pass['normal_pass'] == 1]))

237636
9893
1672


C:\Users\IvovanMiert\AppData\Local\Temp\ipykernel_14948\3397391705.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass['normal_pass'] = 1  # Initialize the column with default values (e.g., 1)


226071


In [143]:
#Succesfullness:
print(df_pass['pass_accurate'].unique())
#Only True and False so that is perfect

[True False]


In [144]:
df_pass.columns

Index(['Unnamed: 0', 'event_id', 'match_id', 'match_period', 'minute',
       'second', 'match_timestamp', 'video_timestamp', 'primary_type',
       'secondary_types',
       ...
       'head_shot', 'whistle', 'shot_after_free_kick', 'acceleration',
       'penalty_foul', 'penalty_goal', 'penalty_conceded_goal', 'third_assist',
       'penalty_save', 'normal_pass'],
      dtype='object', length=143)

In [145]:
df_different_length = df_pass[(df_pass['long_pass'] == 0) & (df_pass['short_or_medium_pass'] == 0)]
print(len(df_different_length))
print(len(df_pass))
#18 rows are going to deleted.
condition_to_remove = (df_pass['long_pass'] == 0) & (df_pass['short_or_medium_pass'] == 0)
df_pass = df_pass[~condition_to_remove]
len(df_pass)

18
237636


237618

In [146]:
print(len(df_pass[(df_pass['forward_pass'] == 0) & (df_pass['back_pass'] == 0) & (df_pass['lateral_pass'] == 0)]))

35189


In [147]:
print(len(df_pass[(df_pass['forward_pass'] == 1) & (df_pass['back_pass'] == 1)]))
condition_to_remove = (df_pass['forward_pass'] == 1) & (df_pass['back_pass'] == 1)
df_pass = df_pass[~condition_to_remove]

68


In [148]:
print(len(df_pass[(df_pass['forward_pass'] == 1) & (df_pass['lateral_pass'] == 1)]))

0


In [149]:
print(len(df_pass[(df_pass['back_pass'] == 1) & (df_pass['lateral_pass'] == 1)]))

0


In [150]:
df_forward = df_pass[df_pass['forward_pass'] == 1]
print(len(df_forward))
print(len(df_forward[(df_forward['pass_angle'] < -45) | (df_forward['pass_angle'] > 45)]))
#print(df_forward['pass_angle'])

78402
185


In [152]:
df_backward = df_pass[df_pass['back_pass'] == 1]
print(len(df_backward))
print(len(df_backward[(df_backward['pass_angle'] > -135) & (df_backward['pass_angle'] < 135)]))
condition = ((df_backward['pass_angle'] >= -135) & (df_backward['pass_angle'] < 135))

36181
123


In [153]:
df_lateral = df_pass[df_pass['lateral_pass'] == 1]
print(len(df_lateral))
print(len(df_lateral[((df_lateral['pass_angle'] < -135) & (df_lateral['pass_angle'] > 135)) | ((df_lateral['pass_angle'] < 45) & (df_lateral['pass_angle'] > -45))]))
print(len(df_lateral[((df_lateral['pass_angle'] <= -135) & (df_lateral['pass_angle'] >= 135)) | ((df_lateral['pass_angle'] <= 45) & (df_lateral['pass_angle'] >= -45))]))

87778
87
1256


In [154]:
df_smaller_12 = df_pass[(df_pass['forward_pass'] == 0) & (df_pass['back_pass'] == 0) & (df_pass['lateral_pass'] == 0)]
print(len(df_smaller_12))
print(len(df_smaller_12[df_smaller_12['pass_length'] > 12]))
print(len(df_smaller_12[(df_smaller_12['pass_angle'] >= -45) & (df_smaller_12['pass_angle'] <= 45)]))
print(len(df_smaller_12[(df_smaller_12['pass_angle'] <= -135) | (df_smaller_12['pass_angle'] >= 135)]))
print(len(df_smaller_12[((df_smaller_12['pass_angle'] > -135) & (df_smaller_12['pass_angle'] < -45)) | ((df_smaller_12['pass_angle'] > 45) & (df_smaller_12['pass_angle'] < 135))]))


35189
132
1155
903
33131


In [129]:
print(df_smaller_12[(df_smaller_12['pass_angle'] >= -45) & (df_smaller_12['pass_angle'] <= 45)])

        Unnamed: 0    event_id  match_id match_period  minute  second  \
393            393  1723218614   5476240           1H      23       3   
423            423  1723218744   5476240           1H      23      48   
1130          1130  1723219399   5476240           2H      58       9   
4185          4185  1720968711   5476242           2H      54      43   
4538          4538  1720969104   5476242           2H      79      41   
...            ...         ...       ...          ...     ...     ...   
492458      492458  2142881780   5476536           2H      92      23   
492588      492588  2139886171   5476537           1H       3      38   
493141      493141  2139886767   5476537           1H      32      26   
494303      494303  2139888158   5476537           2H      94      44   
495127      495127  2140028645   5476539           1H      45       3   

       match_timestamp  video_timestamp primary_type  \
393       00:23:03.016      1384.016317         pass   
423       0

In [167]:
#Dividing the smaller than 12 meter passes over the forward, back and lateral based on their passing degree. 
df_pass.loc[
    (df_pass['forward_pass'] == 0) & 
    (df_pass['back_pass'] == 0) & 
    (df_pass['lateral_pass'] == 0) & 
    (((df_pass['pass_angle'] > -135) & (df_pass['pass_angle'] < -45)) | 
     ((df_pass['pass_angle'] > 45) & (df_pass['pass_angle'] < 135))),
    'lateral_pass'
] = 1

df_pass.loc[
    (df_pass['forward_pass'] == 0) & 
    (df_pass['back_pass'] == 0) & 
    (df_pass['lateral_pass'] == 0) & 
    (df_pass['pass_angle'] >= -45) & 
    (df_pass['pass_angle'] <= 45), 
    'forward_pass'
] = 1

df_pass.loc[
    (df_pass['forward_pass'] == 0) & 
    (df_pass['back_pass'] == 0) & 
    (df_pass['lateral_pass'] == 0) & 
    ((df_pass['pass_angle'] <= -135) | 
     (df_pass['pass_angle'] >= 135)), 
    'back_pass'
] = 1


In [168]:
#Smart/Progressive/Key pass
df_smart = df_pass[df_pass['smart_pass'] == 1]
print(len(df_smart))
df_key = df_pass[df_pass['key_pass'] == 1]
print(len(df_key))
df_progressive = df_pass[df_pass['progressive_pass'] == 1]
print(len(df_progressive))
df_not_progr_key_smart = df_pass[(df_pass['smart_pass'] == 0) & (df_pass['progressive_pass'] == 0) & (df_pass['key_pass'] == 0)]
print(len(df_not_progr_key_smart))

df_key_progressive = df_pass[(df_pass['key_pass'] == 1) & (df_pass['progressive_pass'] == 1)]
print(len(df_key_progressive))
df_key_smart = df_pass[(df_pass['key_pass'] == 1) & (df_pass['smart_pass'] == 1)]
print(len(df_key_smart))
df_progressive_smart = df_pass[(df_pass['progressive_pass'] == 1) & (df_pass['smart_pass'] == 1)]
print(len(df_progressive_smart))

#It can be concluded that the one does not exclude the other. So it can not be classified seperate from each other. New column adding for none of those options

1576
1021
35125
200893
398
144
639


In [169]:
#Into area:
df_deep_completion = df_pass[df_pass['deep_completion'] == 1]
print(len(df_deep_completion))
df_into_penalty_area = df_pass[df_pass['pass_to_penalty_area'] == 1]
print(len(df_into_penalty_area))
df_into_final_third = df_pass[df_pass['pass_to_final_third'] == 1]
print(len(df_into_final_third))

df_deep_completion_and_penalty_area = df_pass[(df_pass['deep_completion'] == 1) & (df_pass['pass_to_penalty_area'] == 1)]
df_deep_completion_and_final_third = df_pass[(df_pass['deep_completion'] == 1) & (df_pass['pass_to_final_third'] == 1)]
df_final_third_and_penalty_area = df_pass[(df_pass['pass_to_final_third'] == 1) & (df_pass['pass_to_penalty_area'] == 1)]
print(len(df_deep_completion_and_penalty_area), len(df_deep_completion_and_final_third), len(df_final_third_and_penalty_area))

#They don't exclude each other. 


4027
5781
27565
2604 458 1653


In [170]:
df_pass['accurate'] = 0
df_pass['not_accurate'] = 0
# Set the values based on the 'pass_accurate' column
# Set values based on 'pass_accurate' column
df_pass.loc[(df_pass['pass_accurate'] == True), 'accurate'] = 1
df_pass.loc[(df_pass['pass_accurate'] == False), 'not_accurate'] = 1

In [172]:
def select_minimum_clips(df, primary_type, attribute_sets, max_difference_per_set=500, min_clips_per_attribute=1050, chunk_size=10000):
    # Filter the DataFrame for the specified primary type
    primary_df = df 
    # Dictionary to hold the selected clips for each attribute set
    selected_clips_dict = {attr_set: [] for attr_set in attribute_sets}
    
    # Function to check if all attribute sets have enough clips
    def all_attribute_sets_satisfied():
        return all(len(clips) >= min_clips_per_attribute * len(attr_set) for attr_set, clips in selected_clips_dict.items())
    
    # Iterate through the DataFrame once per attribute set
    for attr_set in attribute_sets:
        attr_clips = []
        for attr in attr_set:
            # Extract clips where the specific column (attr) is True (1)
            attr_clips.extend(primary_df[primary_df[attr] == 1]['event_id'].tolist())
        
        # Shuffle the clips to randomize selection
        np.random.shuffle(attr_clips)
        
        # Select exactly min_clips_per_attribute clips for this attribute set
        selected_clips_dict[attr_set] = attr_clips[:min_clips_per_attribute]
        
        # Check if all attribute sets have enough clips after each set
        if all_attribute_sets_satisfied():
            break
    
    # Combine all selected clips ensuring no duplicates
    selected_clips = set()
    for clips in selected_clips_dict.values():
        selected_clips.update(clips)

    return selected_clips

In [173]:
# Shuffle the rows of the DataFrame
df_shuffled = df_pass.sample(frac=1).reset_index(drop=True)

# Define primary type and attribute sets as tuples
# Define primary type and attribute sets as tuples
primary_type = 'pass'
attribute_sets = [
    ('forward_pass', 'back_pass', 'lateral_pass'),
    ('head_pass', 'hand_pass', 'normal_pass'),
    ('long_pass', 'short_or_medium_pass'),
    ('smart_pass', 'key_pass', 'progressive_pass', 'normal_pass'),
    ('deep_completion', 'pass_to_final_third', 'pass_to_penalty_area'),
    ('accurate', 'not_accurate')
]

# Select minimum clips to cover all classifications
selected_clips = select_minimum_clips(df_shuffled, primary_type, attribute_sets)
print(f"Selected clip IDs: {selected_clips}")
print(f"Number of selected clips: {len(selected_clips)}")

Selected clip IDs: {2093875211, 1770815508, 1724481557, 1777958943, 1777958946, 1777958947, 1770815552, 2130903106, 1785888840, 1744273501, 2093875297, 1810759789, 1777959022, 1835171951, 2130903159, 1810759800, 2093875323, 2100199547, 1770815615, 1777959043, 1819115655, 1835171979, 2130903202, 1785888935, 1813545131, 1819115694, 1835172017, 1813545137, 2090598587, 2090598588, 1911881919, 1874329807, 1911881937, 1819115750, 1813545192, 2090598632, 1724481771, 1835172102, 1874329865, 2130903306, 1911881999, 2130903328, 1744273702, 1911882028, 1797030191, 1742405938, 1911882037, 2100003128, 1742405945, 1770815814, 1777959239, 1785889099, 1911882072, 1835172193, 1744273781, 1861058941, 1777959301, 1813545353, 1785889162, 1797030286, 2100199823, 1861058960, 2130903450, 2090598811, 1744273827, 1797030308, 1819115941, 1811579306, 1777959343, 1911882161, 2100003253, 1813545410, 2090598857, 1813545421, 1744273881, 1744273897, 1770815977, 1829896695, 1835172346, 1785889276, 2100003324, 17858892

In [174]:
df_selected_clips = df_shuffled[df_shuffled['event_id'].isin(selected_clips)].reset_index(drop=True)


In [175]:
df_selected_clips.to_csv('D:\\GITHUB\\Data_CEP\\Exploration\\selected_clips_pass.csv', index=False)

In [177]:
list = ['forward_pass', 'back_pass', 'lateral_pass', 'head_pass', 'hand_pass', 'normal_pass','long_pass', 'short_or_medium_pass','smart_pass', 'key_pass', 'progressive_pass', 'normal_pass', 'deep_completion', 'pass_to_final_third', 'pass_to_penalty_area', 'accurate', 'not_accurate']
for att in list:
    print(len(df_selected_clips[df_selected_clips[att] ==1]))

2492
818
2921
255
39
5937
896
5335
102
55
1359
5937
275
1446
407
5252
979
